In [1]:
# py-sc-fermi

import timeit

start = timeit.default_timer()

from py_sc_fermi.defect_system import DefectSystem
from py_sc_fermi.defect_species import DefectSpecies
from py_sc_fermi.defect_charge_state import FrozenDefectChargeState
from py_sc_fermi.inputs import inputs_from_files

import numpy as np

In [2]:
example_dir = 'example_inputs'

unitcell_filename = f'{example_dir}/unitcell.dat'
totdos_filename = f'{example_dir}/totdos.dat'
input_fermi_filename = f'{example_dir}/input-fermi.dat'

inputs = inputs_from_files(unitcell_filename=unitcell_filename, 
                           input_fermi_filename=input_fermi_filename, 
                           totdos_filename=totdos_filename)

Volume of cell: 57.78888409009906 A^3
Found non-spin polarised system...
Number of electrons in system: 18
Energy gap of system: 0.8084 eV
Temperature: 100.0 K
Number of defect species: 2


In [3]:
defect_system = DefectSystem( defect_species=inputs['defect_species'],
                              volume=inputs['volume'],
                              nelect=inputs['nelect'],
                              edos=inputs['edos'],
                              dos=inputs['dos'],
                              egap=inputs['egap'], 
                              temperature=300.0 )

defect_system

DefectSystem
  nelect: 18 e
  egap:   0.8084 eV
  volume: 57.78888409009906 A^3
  temperature: 300.0 K

Contains defect species:

V_Ga, nsites=1
  q=+0.0, e=2.4451, deg=1
  q=-1.0, e=0.0265, deg=1
  q=-2.0, e=2.3469, deg=1
  q=-3.0, e=2.7146, deg=1

Ga_Sb, nsites=1
  q=+0.0, e=2.2649, deg=1
  q=-1.0, e=2.0937, deg=1
  q=-2.0, e=2.2527, deg=1

In [4]:
from collections import Counter
nfrozen = 1
namef = ['V_Ga']
concf = [0.3285677364522e20]
# check frozen defects are included in the original list of defects
for name in namef:
    if name not in defect_system.defect_species_names:
        raise ValueError((f'ERROR: fixed concentration defect "{name}"'+
                           'is not in list of original defects!\n'+
                           'Please add new frozen defects as fixed charge defects in'
                           'a separate list - otherwise a solution cannot be found!!'))
# Check if any defects have been assigned multiple fixed concentrations
c = Counter(namef)
for n, count in c.items():
    if count > 1:
        raise ValueError(f'ERROR: defect "{n}"has multiple fixed concentrations!!')
for n, c in zip(namef, concf):
    defect_system.defect_species_by_name(n).fixed_concentration = c * defect_system.volume / 1e24
    
defect_system

DefectSystem
  nelect: 18 e
  egap:   0.8084 eV
  volume: 57.78888409009906 A^3
  temperature: 300.0 K

Contains defect species:

V_Ga, nsites=1
fixed [c] = 0.0018987562837582401 cm^-3
  q=+0.0, e=2.4451, deg=1
  q=-1.0, e=0.0265, deg=1
  q=-2.0, e=2.3469, deg=1
  q=-3.0, e=2.7146, deg=1

Ga_Sb, nsites=1
  q=+0.0, e=2.2649, deg=1
  q=-1.0, e=2.0937, deg=1
  q=-2.0, e=2.2527, deg=1

In [5]:
ficharg = False
fixconch = False
namefn = ['V_Ga', 'Ga_i']
chargefn = [-1.0, 1.0]
concfn = [0.19e19, 0.5e20]
if 0 in chargefn:
    checkfixchg = True
else:
    checkfixchg = False
for n,q,c  in zip( namefn, chargefn, concfn ):
    print(c)
    c = c * defect_system.volume / 1e24
    frozen_charge_state = FrozenDefectChargeState(charge=q, concentration=c)
    if n in defect_system.defect_species_names:     
        defect_system.defect_species_by_name(n).charge_states[q] = frozen_charge_state
    else:
        ds = DefectSpecies(name=n, nsites=1, charge_states=[frozen_charge_state])
        defect_system.defect_species.append(ds)

defect_system.check_concentrations()
defect_system

1.9e+18
5e+19


DefectSystem
  nelect: 18 e
  egap:   0.8084 eV
  volume: 57.78888409009906 A^3
  temperature: 300.0 K

Contains defect species:

V_Ga, nsites=1
fixed [c] = 0.0018987562837582401 cm^-3
  q=+0.0, e=2.4451, deg=1
  q=-1.0, [c]=0.00010979887977118821
  q=-2.0, e=2.3469, deg=1
  q=-3.0, e=2.7146, deg=1

Ga_Sb, nsites=1
  q=+0.0, e=2.2649, deg=1
  q=-1.0, e=2.0937, deg=1
  q=-2.0, e=2.2527, deg=1

Ga_i, nsites=1
  q=+1.0, [c]=0.002889444204504953